### importing libraries

In [1]:
from google.cloud import storage
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
pd.set_option('display.max_columns', None)

import os
import csv
import json
import gzip
import glob

import haversine as hs

from tqdm import tqdm

from datetime import datetime, timedelta

### calling the google cloud storage bucket object and pushing not yet uploaded files

In [2]:
# instantiating the google cloud storage client using the credentials
storage_client = storage.Client.from_service_account_json('ds_credentials_edit.json')

In [3]:
# declaring the bucket name from which the date needs to be fetched
bucket_name = 'ds_marketplace-animall_scraped_data'

# get the bucket object from the storage_client
bucket = storage_client.get_bucket(bucket_name)

In [4]:
# declaring empty list to store the filenames that needs to be downloaded from the bucket
blob_file_names = []

# get names of all the blobs in the bucket
for blob_ in bucket.list_blobs():
    blob_file_names.append(blob_.name)

In [5]:
# crawling through folder to get all the file names with local directory
local_path = "/home/customer/Narada/ScrapedData"
all_folders = glob.glob(local_path + '/**')
files_uploaded_dir_remote = []
files_uploaded_dir_local = []

while len(all_folders)!=0:
    local_file = all_folders.pop()
    if not os.path.isfile(local_file):
        all_folders.extend(glob.glob(local_file + '/**'))
    else:
        files_uploaded_dir_remote.append(local_file[22:])
        files_uploaded_dir_local.append(local_file)

In [6]:
# creating dataframe for the directory path and file_names
all_file_dir = pd.DataFrame({'remote':files_uploaded_dir_remote,\
                         'local':files_uploaded_dir_local
                        })

In [7]:
# filtering out the files that need to pushed to google cloud storage
file_dir = all_file_dir[~all_file_dir['remote'].isin(blob_file_names)].reset_index(drop=True)

In [8]:
# count of file to be uploaded to google cloud storage
len(file_dir)

6

In [9]:
# iterating and trying to upload all data programatically
for i in tqdm(range(len(file_dir))):
    # Name of the object to be stored in the bucket
    object_name_in_bucket = bucket.blob(file_dir['remote'].iloc[i])
    
    # Name of the object in local file system
    object_name_in_bucket.upload_from_filename(file_dir['local'].iloc[i])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:22<00:00,  3.74s/it]


### segregating and labelling the files into "Animal Details", "Seller Details" and "Resource Details"

In [13]:
# segragating "Animal Details", "Resources" and "Seller Info" files
def labelling_files(x):
    # handling bad files
    if len(x.split(".")[0])>5:
        # splitting "file name" into 'sub_file_name_tokens'
        sub_file_name_tokens = x.split(".")[0].split("_")

        # getting 'file_type' - AD/seller/resource
        file_type = sub_file_name_tokens[-1]

        # getting 'date_scraped'
        date_scraped = sub_file_name_tokens[-2]

        # getting 'state' (cross checking)
        state = sub_file_name_tokens[0]
        
        # returns data as tuple
        return [file_type, date_scraped, state]

    else:
        return ['NA', 'NA', 'NA']

In [14]:
# getting file_names to segregate and process files
file_names = [item.split("/")[-1] for item in files_uploaded_dir_local]

In [15]:
# dataFrame with details of all the files in our 'ScrapedData' for 'classified' business
file_df = pd.DataFrame({'file_names':file_names,\
                        'location':files_uploaded_dir_local
                       })

In [16]:
# adding metaData to the 'file_df' DataFrame
file_df[['file_type','date_scraped','state']] = file_df['file_names'].apply(lambda x:labelling_files(x)).tolist()

# dropping 'NA' rows
file_df = file_df[file_df['state']!='NA']

# converting 'date_scraped' to datetime format, and sorting dataFrame
file_df['date_scraped'] = pd.to_datetime(file_df['date_scraped'])
file_df.sort_values(by='date_scraped', inplace=True)
file_df.reset_index(drop=True, inplace=True)

In [17]:
# fetching most recent sample "classified-animal_details" excel file
AD_files = file_df[file_df['file_type']=='AD'].reset_index(drop=True)

# fetching most recent sample "classified-seller" excel file
seller_files = file_df[file_df['file_type']=='seller'].reset_index(drop=True)

# fetching most recent sample "classified-resource" excel file
resource_files = file_df[file_df['file_type']=='resource'].reset_index(drop=True)

### all ANIMAL details files cattle movement analysis

In [18]:
# function to concatenate all "Animal Details" excel files
def get_animal_details(files_df_):
    # declaring empty list to concat all dataFrames
    dflist = []
    
    # looping and concatenating
    for i in range(len(files_df_)):#[-10] to take most recent 10 days scraped data
        
        # file_path of all excel files
        file_path_ = files_df_['location'].iloc[i]

        # reading the file        
        df1 = pd.read_excel(file_path_)
        df1['date_scraped'] = files_df_['date_scraped'].iloc[i]
        df1['date_scraped'] = pd.to_datetime(df1['date_scraped'])
        df1['listing_state'] = files_df_['state'].iloc[i]
        df1['file_age'] = df1['date_scraped'].apply(lambda x: int((datetime.today() - x).days)+1)
        dflist.append(df1)
        
    # concatenating and dropping index
    concatenated_df = pd.concat(dflist).reset_index().drop('index',axis=1)
    
    # renaming columns
    concatenated_df.rename(columns={'_id':'cattle_id','userId':'seller_id', 'seller':'seller_name', 'state':'listing_status'}, inplace=True)
    
    # returning the concatenated dataFrame
    return concatenated_df

In [19]:
# calling the dataFrames concatenating function
all_animal_details_df = get_animal_details(AD_files)

In [20]:
# dropping off "PENDING" and "STA" listing_status - only 2 such instances
all_animal_details_df = all_animal_details_df[all_animal_details_df['listing_status']=='ACTIVE']

# getting all unique 'cattle_id'
all_animal_details_df['cattle_id'] = all_animal_details_df['cattle_id'].astype(str)
unique_cattle_ids = list(all_animal_details_df['cattle_id'].unique())

# converting columns into 'datetime' format
all_animal_details_df['publishedOn'] = pd.to_datetime(all_animal_details_df['publishedOn'])
all_animal_details_df['date_scraped'] = pd.to_datetime(all_animal_details_df['date_scraped'])

# getting the 'last_scraped_date'
last_scraped_date = all_animal_details_df['date_scraped'].max()

In [21]:
# replacing NaN values
all_animal_details_df['description'].fillna("-", inplace=True)
all_animal_details_df['deliveredBefore'].fillna(-999, inplace=True)
all_animal_details_df['hasDelivered'].fillna(-999, inplace=True)
all_animal_details_df['isPregnant'].fillna(-999, inplace=True)
all_animal_details_df['pregnancyMonth'].fillna(-999, inplace=True)
all_animal_details_df['calf'].fillna('-', inplace=True)
all_animal_details_df['partnerId'].fillna(-999, inplace=True)
all_animal_details_df['hasContacted'].fillna(-999, inplace=True)
all_animal_details_df['breed'].fillna("-", inplace=True)

# replacing int 0 value with str
all_animal_details_df['calf'].replace(0,'-', inplace=True)
all_animal_details_df['breed'].replace(0,'-', inplace=True)

# dropping redundant columns
all_animal_details_df.drop(columns=['description'], inplace=True)
all_animal_details_df.drop(columns=['district'], inplace=True)
all_animal_details_df.drop(columns=['dynamicLink'], inplace=True)

In [22]:
all_animal_details_df.shape

(458643, 35)

In [23]:
# adding date of results to all_animal_details csv file_name
all_animal_details_file_name = "processed_data_files/all_animals_details_df_"+str(datetime.today().date())+".parquet"

# saving to disk
all_animal_details_df.to_parquet(all_animal_details_file_name, index=False)

In [24]:
# sdf_ = pd.read_parquet('processed_data_files/all_animals_details_df_2022-09-21.parquet', engine='fastparquet')

In [25]:
# sdf_.head()

In [26]:
# grouped based on "cattle_id", sorted by "date_scraped", taking the last occurence
cattle_sold = pd.DataFrame(all_animal_details_df.sort_values(by='date_scraped').groupby('cattle_id').tail(1)).reset_index(drop=True)


In [27]:
# function to get listing_status(Sold/Unsold/Fresh_listing) an the no. of days taken to sell item if sold
def get_listing_status(x, last_scraped_dt):
    # "UnSold" listing
    if ((last_scraped_dt-pd.to_datetime(x['date_scraped'])).days)==0:
        days_to_sell = -999
        sold_status = 'UNSOLD'
    # "Fresh" listing
    elif ((last_scraped_dt-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)==0:
        days_to_sell = -999
        sold_status = 'FRESH_LISTING'
    # "De-listed" listing
    elif int((pd.to_datetime(x['date_scraped'])-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)+1==29:
        days_to_sell = -999
        sold_status = 'DELISTED_LISTING'
    # "Sold" listing
    else:
        days_to_sell = int((pd.to_datetime(x['date_scraped'])-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)+1
        sold_status = 'Yes'
     
    # returning the no. of "days_taken_to_sell" and the final "sold_status"
    return [days_to_sell, sold_status]

In [28]:
# applying the "get_listing_status" function
cattle_sold[['days_to_sell', 'sold_status']] = cattle_sold.apply(lambda y:get_listing_status(y, last_scraped_date),axis=1).tolist()

In [29]:
cattle_sold.shape

(94191, 37)

In [30]:
# adding date of results to cattle_sold csv file_name
cattle_sold_file_name = "processed_data_files/cattle_sold_df_"+str(datetime.today().date())+".parquet"

# saving to disk
cattle_sold.to_parquet(cattle_sold_file_name, index=False)

### all SELLER details files contact number mapping

In [31]:
# function to concatenate all "Seller" excel files
def get_seller_details(files_df_):
    # declaring empty list to concat all dataFrames
    dflist = []
    
    # looping and concatenating
    for i in range(len(files_df_)):#[-10] to take most recent 10 days scraped data
        
        # file_path of all excel files
        file_path_ = files_df_['location'].iloc[i]
        
        # reading the file
        df1 = pd.read_excel(file_path_)
        df1['date_scraped'] = files_df_['date_scraped'].iloc[i]
        df1['date_scraped'] = pd.to_datetime(df1['date_scraped'])
        df1['listing_state'] = files_df_['state'].iloc[i]
        df1['file_age'] = df1['date_scraped'].apply(lambda x: int((datetime.today() - x).days)+1)
        dflist.append(df1)
        
    # concatenating and dropping index
    concatenated_df = pd.concat(dflist).reset_index().drop('index',axis=1)
    
    # renaming columns
    concatenated_df.rename(columns={'_id':'cattle_id','userId':'seller_id', 'seller':'seller_name', 'state':'listing_status'}, inplace=True)
    
    # returning the concatenated dataFrame
    return concatenated_df

In [58]:
# calling the dataFrames concatenating function
all_seller_details_df = get_seller_details(seller_files)

In [66]:
# tagging and filtering out 10 digit seller_id
all_seller_details_df['contact_number_y/n'] = all_seller_details_df['id'].apply(lambda x: "Yes" \
                                                                                if(len(str(x))==10) else "No")

In [69]:
all_contacts = all_seller_details_df[all_seller_details_df['contact_number_y/n'] == 'Yes'].reset_index(drop=True)

In [71]:
all_contacts.shape

(107272, 14)

In [70]:
all_contacts.head()

,cattle_id,id,name,nameEn,profileImage,description,slug,createdAt,appVersion,isVerified,date_scraped,listing_state,file_age,contact_number_y/n
0,62dcbddee147a6000a6eb346,8433297856,ParvindChaudhary,ParvindChaudhary,NaN,NaN,parvindchaudhary-neiiwxogf8,2020-08-14T04:57:14.000Z,1.9.9.5,False,2022-07-25,DELHI,59,Yes
1,62dba89a4845ed000aba4585,9760980193,Museer khan,Museer khan,NaN,NaN,museer-khan-bwhgh7l7av,2020-09-13T05:06:30.000Z,1.9.9.5,False,2022-07-25,DELHI,59,Yes
2,62dce5ffd94b17000af5c673,8910701752,mahesh kumar,mahesh kumar,https://static-assets.animall.in/profile-image...,NaN,mahesh-kumar-ftyxw0fsue,2020-08-13T02:16:23.000Z,1.9.9.5,False,2022-07-25,DELHI,59,Yes
3,62dd01a447263e000a6a254e,6398108982,Navbahr dhaka,Navbahr dhaka,https://static-assets.animall.in/profile-image...,शुगर मिल बागपत मेरठ रोड निकट मैन गेट,navbahr-dhaka-ltnj2ye47j,2020-10-03T16:12:17.000Z,1.9.9.5,False,2022-07-25,DELHI,59,Yes
4,62d91bad179568000a0cd296,7988165861,Tushar,Tushar,https://static-assets.animall.in/profile-image...,NaN,tushar-bm6hqpu3lo,2020-10-26T00:56:01.000Z,1.9.9.5,False,2022-07-25,DELHI,59,Yes


In [38]:
# dropping redundant data rows, reatining only 10 digit values
all_seller_details_df['id'] = all_seller_details_df['id'].astype(str)

nums_ = [ row['id'] for _, row in all_seller_details_df.iterrows() if len(row['id']) == 10]
all_seller_details_df = all_seller_details_df[all_seller_details_df['id'].isin(nums_)]
all_seller_details_df.reset_index(drop=True, inplace=True)

In [57]:
len(cattle_sold), len(all_seller_details_df)

(94191, 107272)

In [55]:
cattle_sold[cattle_sold['seller_id']=='9813968303']

,cattle_id,seller_id,distance,rating,viewCount,callCount,recency,_score,long,lat,gender,listing_status,highestMilk,seller_name,locationName,isNegotiable,publishedOn,currentMilk,deliveredBefore,hasDelivered,isPregnant,pregnancyMonth,price,age,animalType,breed,calf,lactation,partnerId,hasContacted,type,coordinates,date_scraped,listing_state,file_age,days_to_sell,sold_status


In [50]:
all_seller_details_df.groupby(['id']).count().reset_index().sort_values(by=['cattle_id'], ascending=False)

,id,cattle_id,name,nameEn,profileImage,description,slug,createdAt,appVersion,isVerified,date_scraped,listing_state,file_age
10219,9813968303,242,242,242,242,242,242,242,242,242,242,242,242
9348,9729304332,188,188,188,188,188,188,188,188,188,188,188,188
1062,7017158702,181,181,181,181,0,181,181,181,181,181,181,181
9352,9729355326,161,161,161,161,161,161,161,161,161,161,161,161
9637,9779189523,136,136,136,0,0,136,136,136,136,136,136,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9036,9694016778,1,1,1,1,0,1,1,1,1,1,1,1
9033,9694000505,1,1,1,0,0,1,1,1,1,1,1,1
9032,9693394910,1,1,1,0,0,1,1,1,1,1,1,1
9022,9691006469,1,1,1,0,0,1,1,1,1,1,1,1


In [51]:
all_seller_details_df[all_seller_details_df['id']=='9813968303']

,cattle_id,id,name,nameEn,profileImage,description,slug,createdAt,appVersion,isVerified,date_scraped,listing_state,file_age
192,62dded30458b97000b4b7fb9,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,2.1.6,False,2022-07-25,HARYANA,59
515,62ddedbc8f3750000ad526f7,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,2.1.6,False,2022-07-25,HARYANA,59
548,62db94bc7deed3000ab2c7ab,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,2.1.6,False,2022-07-25,HARYANA,59
1097,62e3cfdbed0f3f000b5caa2b,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.6,False,2022-08-02,HARYANA,51
2832,62eb60c80896b4000a8f0e0f,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.6,False,2022-08-05,RAJASTHAN,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106582,63271e814a576f000a237dcf,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.7,False,2022-09-21,RAJASTHAN,1
106583,632ad2acc34458000b1e4c84,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.7,False,2022-09-21,RAJASTHAN,1
106913,6329679b51d518000ac2bf76,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.7,False,2022-09-21,HARYANA,1
106915,63271e814a576f000a237dcf,9813968303,Vijay Saini,Vijay Saini,https://static-assets.animall.in/profile-image...,Jai shree shyam,vijay-saini-j32nzl1o2o,2020-09-14T14:44:22.000Z,1.9.9.7,False,2022-09-21,HARYANA,1


In [ ]:
all_seller_details_df.drop_duplicates(subset=['id'],inplace=True)

In [ ]:
all_seller_details_df = all_seller_details_df[['id','name']]

In [ ]:
all_seller_details_df.rename(columns={'id':'contact_number'}, inplace=True)

In [ ]:
all_seller_details_df = all_seller_details_df.drop_duplicates(subset=['name']).reset_index(drop=True)

In [ ]:
all_seller_details_df.shape

In [ ]:
all_seller_details_df['contact_number'] = all_seller_details_df['contact_number'].astype(int)

In [ ]:
# adding date of results to all_animal_details csv file_name
all_seller_details_file_name = "processed_data_files/all_seller_details_df_"+str(datetime.today().date())+".parquet"

# saving to disk
all_seller_details_df.to_parquet(all_seller_details_file_name, index=False)

In [ ]:
# # adding date of results to all_animal_details csv file_name
# all_seller_details_file_name = "processed_data_files/all_seller_details_df_"+str(datetime.today().date())+".csv"

# # saving to disk
# all_seller_details_df.to_csv(all_seller_details_file_name, index=False)

### all cattles pincode mapping - gmap_lat_long

In [ ]:
# reading the current gmap_lat_long_mapping csv
gmap_lat_long_mapping_df = pd.read_csv("processed_data_files/gmap_lat_long_mapping_df_2022-09-21.csv")

In [ ]:
gmap_lat_long_mapping_df.shape

In [ ]:
cattle_sold.shape

In [ ]:
# filtering out 'cattle_ids' that need to be mapped
tbd_df = cattle_sold[~cattle_sold['cattle_id'].isin(gmap_lat_long_mapping_df['cattle_id'].tolist())].reset_index(drop=True)

In [ ]:
tbd_df.shape

In [ ]:
# reading the GMAP lat_long data
lat_long_df = pd.read_csv("GMAP_full.csv")

In [ ]:
for i, row in tqdm(tbd_df.iterrows()):
    lat = float(row['lat'])
    long = float(row['long'])
    source = (lat,long)
    distance_matrixs = {}
    for j, row_ in lat_long_df.iterrows():
        glat = float(row_['Lat_GMAP'])
        glong = float(row_['Long_GMAP'])
        destination = (glat,glong)
        distance = hs.haversine(source,destination)
        distance_matrixs[j] = distance

    distance_matrixs = sorted(distance_matrixs.items(), key=lambda x:x[1])
    distance_mats = dict(distance_matrixs)
    max_idx = list(distance_mats.keys())[0]

    tbd_df.loc[i,'idx'] = max_idx
    tbd_df.loc[i,'GLong'] = lat_long_df.iloc[max_idx]['Long_GMAP']
    tbd_df.loc[i,'GLat'] = lat_long_df.iloc[max_idx]['Lat_GMAP']
    tbd_df.loc[i,'Pincode'] = lat_long_df.iloc[max_idx]['Pincode']
    tbd_df.loc[i,'Distance'] = distance_mats[max_idx]*1.2

In [ ]:
# columns to be retained from tbd_df
col_names = gmap_lat_long_mapping_df.columns.tolist()

In [ ]:
# concatenated "gmap_mapping_result" dataframe
gmap_mapping_result = pd.concat([gmap_lat_long_mapping_df,tbd_df[col_names]],axis=0).reset_index(drop=True)

In [ ]:
# # adding date of results to all_animal_details csv file_name
# gmap_lat_long_file_name = "processed_data_files/gmap_lat_long_mapping_df_"+str(datetime.today().date())+".csv"

# # saving to disk
# gmap_lat_long_mapping_df.to_csv(gmap_lat_long_file_name, index=False)

### indian postal code pincode.csv

In [ ]:
# reading csv
pincode_df = pd.read_csv("pincode.csv", low_memory=False)

In [ ]:
pincode_df.shape

### adding features for reports/analysis

#### contacts mapping

In [ ]:
contacts_df = all_seller_details_df

In [ ]:
cattle_sold = cattle_sold.merge(contacts_df, left_on='seller_name', right_on='name', how='left').fillna(-999)

In [ ]:
cattle_sold.drop(columns=['name'], inplace=True)

#### gmap lat long mapping

In [ ]:
gmap_lat_long_df = gmap_lat_long_mapping_df[['cattle_id','GLong','GLat','Pincode']]

In [ ]:
gmap_lat_long_df = gmap_lat_long_df.rename(columns={'Pincode':'gmap_pincode',\
                                 'GLong':'gmap_long',\
                                 'GLat':'gmap_lat',\
                                })

In [ ]:
cattle_sold = cattle_sold.merge(gmap_lat_long_df, left_on='cattle_id', right_on='cattle_id', how='left').fillna(-999)

In [ ]:
cattle_sold['gmap_pincode'] = cattle_sold['gmap_pincode'].astype(int)

In [ ]:
cattle_sold.shape

#### pincode state and district mapping

In [ ]:
pincodes_df = pincode_df[['Pincode','District','StateName']]

In [ ]:
pincodes_df = pincodes_df.rename(columns={'Pincode':'gmap_pincode',\
                            'District':'gmap_district',\
                            'StateName':'gmap_state',\
                           })

In [ ]:
pincodes_df = pincodes_df.drop_duplicates(subset=['gmap_pincode']).reset_index(drop=True)

In [ ]:
pincodes_df.shape

In [ ]:
cattle_sold = cattle_sold.merge(pincodes_df, left_on='gmap_pincode', right_on='gmap_pincode', how='left').fillna('-')

In [ ]:
cattle_sold = cattle_sold.reset_index(drop=True)

In [ ]:
cattle_sold.shape

#### adding week names and date features

In [ ]:
cattle_sold['month_of_sale'] = cattle_sold['date_scraped'].dt.month_name()
cattle_sold['week_of_sale'] = (cattle_sold['date_scraped']+pd.DateOffset(1)).dt.isocalendar().week
cattle_sold['year_of_sale'] = cattle_sold['date_scraped'].dt.year
cattle_sold['weekday'] = cattle_sold['date_scraped'].dt.strftime('%A')
cattle_sold['week_start_date'] = cattle_sold['date_scraped'].apply(lambda x: (x - timedelta(days=x.weekday()) - pd.DateOffset(1)).strftime('%d %b'))
cattle_sold['week_end_date'] = cattle_sold['date_scraped'].apply(lambda x: (x - timedelta(days=x.weekday()) - pd.DateOffset(1) + timedelta(days=6)).strftime('%d %b'))
cattle_sold['sale_week'] = cattle_sold.apply(lambda x: str(x['week_start_date']) + " - " + str(x['week_end_date']), axis=1)
cattle_sold['sale_day'] = cattle_sold['date_scraped'].apply(lambda x: str(x.strftime('%d %b')) + ": " + str(x.strftime('%a')))


#### renaming and standardizing the column names

In [ ]:
# making a copy of the dataframe
data = cattle_sold.copy()

In [ ]:
# standardizing column names for google cloud storage
col_names_mapping = {'viewCount':'view_count_animall',\
                     'callCount':'call_count_animall',\
                     '_score':'score',\
                     'highestMilk':'highest_milk',\
                     'locationName':'location_name',\
                     'isNegotiable':'is_negotiable',\
                     'publishedOn':'published_on',\
                     'currentMilk':'current_milk',\
                     'deliveredBefore':'delivered_before',\
                     'hasDelivered':'has_delivered',\
                     'isPregnant':'is_pregnant',\
                     'pregnancyMonth':'pregnancy_month',\
                     'age':'cattle_age',\
                     'animalType':'animal_type',\
                     'partnerId':'partner_id',\
                     'hasContacted':'has_contacted',\
                    }

In [ ]:
# renaming columns
data = data.rename(columns=col_names_mapping)

#### standardizing the column data types

In [ ]:
data['distance'] = data['distance'].astype(float)
data['rating'] = data['rating'].astype(float)
data['view_count_animall'] = data['view_count_animall'].astype(int)
data['call_count_animall'] = data['call_count_animall'].astype(int)
data['recency'] = data['recency'].astype(int)
data['score'] = data['score'].astype(float)
data['long'] = data['long'].astype(float)
data['lat'] = data['lat'].astype(float)
data['highest_milk'] = data['highest_milk'].astype(float)
data['is_negotiable'] = data['is_negotiable'].astype(int)
data['published_on'] = pd.to_datetime(data['published_on'])
data['current_milk'] = data['current_milk'].astype(float)
data['delivered_before'] = data['delivered_before'].apply(lambda x: float(x) if x!='-' else x)
data['has_delivered'] = data['has_delivered'].apply(lambda x: float(x) if x!='-' else x)
data['is_pregnant'] = data['is_pregnant'].apply(lambda x: float(x) if x!='-' else x)
data['pregnancy_month'] = data['pregnancy_month'].apply(lambda x: float(x) if x!='-' else x)
data['price'] = data['price'].astype(float)
data['cattle_age'] = data['cattle_age'].astype(float)
data['lactation'] = data['lactation'].astype(int)
data['date_scraped'] = pd.to_datetime(data['date_scraped'])
data['file_age'] = data['file_age'].astype(int)
data['days_to_sell'] = data['days_to_sell'].apply(lambda x: int(x) if len(str(x))<=2 else x)
data['week_of_sale'] = data['week_of_sale'].apply(lambda x: int(x) if x!='UNSOLD' else x)
data['year_of_sale'] = data['year_of_sale'].apply(lambda x: int(x) if x!='UNSOLD' else x)
data['gmap_pincode'] = data['gmap_pincode'].apply(lambda x: int(x) if x!='-' else x)
data['contact_number'] = data['contact_number'].apply(lambda x: int(x) if x not in ['NA','-'] else x)

### converting to csv

In [ ]:
data.to_csv('furnished_data/cattle_sold_data.csv', index=False)
data.to_csv('furnished_data/cattle_sold_no_header.csv', index=False, header=False)

### converting to parquet

In [ ]:
data.to_parquet('furnished_data/cattle_sold_data.parquet', index=False)

### pushing to google cloud storage bucket

In [ ]:
# declaring the bucket name from which the date needs to be fetched
bucket_name = 'marketplace_animall_reporting'

# get the bucket object from the storage_client
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
# Name of the object to be stored in the bucket
object_name_in_bucket = bucket.blob('cattle_sold.parquet')

# local folder name
local_folder = os.getcwd()

# local file location
filename = "%s/%s" % (local_folder, 'furnished_data/cattle_sold_data.parquet')

# Name of the object in local file system
object_name_in_bucket.upload_from_filename(filename)